In [1]:
%cd /home/ducpham/Documents/LightRAG
# move to project

/home/ducpham/Documents/LightRAG


In [ ]:
# init global config for classes 

from config import ConfigParams
from dataclasses import asdict

config = asdict(ConfigParams())
for k, v in config.items():
    print(f"Key: {k}  -> Value: {v}")

In [3]:
import json

with open("./testing/data/entity.json", mode='r') as f:
    all_entities_data = json.load(f)

In [4]:
all_entities_data[1]

{'node_id': 'Mikasa',
 'node_data': {'entity_type': 'person',
  'entity_desc': 'Mikasa là bạn của Eren và Armin, cậu được nhận nuôi bời gia đình của Eren',
  'entity_source_id': '2'}}

In [5]:
from lightrag.vectorstore import NanoVectorStorage
from lightrag.embeddings import openai_embedding
from lightrag.utils import compute_mdhash_id
from dataclasses import field


entity_vdb = NanoVectorStorage(
    namespace="Test_LightRAG",
    global_config=config,
    embedding_func=openai_embedding,
    meta_fields={"entity_name"}
)


INFO:nano-vectordb:Load (2, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': 'my_working/vdb_Test_LightRAG.db'} 2 data


In [6]:

all_entities_data = [
    {
        "entity_name": "Eren", 
        "entity_desc": "Eren Yeager là nhân vật chính, có khả năng biến thành Titan"
    },
    {
        "entity_name": "Mikasa",
        "entity_desc": "Mikasa là bạn của Eren và Armin, cậu được nhận nuôi bời gia đình của Eren"
    }
]

if entity_vdb is not None: # for entity
    data_for_vdb = {
        compute_mdhash_id(content=dp['entity_name'], prefix="ent-"): {
            "entity_name": dp['entity_name'],
            "content": dp['entity_name'] + dp['entity_desc']
        }
        for dp in all_entities_data
    }
    await entity_vdb.upsert(data=data_for_vdb)



INFO:lightrag:Inserting 2 vectors to Test_LightRAG
Generating vector embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.23batch/s]


In [7]:
await entity_vdb.index_done_callback()

In [8]:
entity_vdb._client.get(compute_mdhash_id("Eren", prefix="ent-"))

[{'__id__': 'ent-44448f65f1126a30f11b0b7c9809fdb9', 'entity_name': 'Eren'}]